<a href="https://colab.research.google.com/github/jbloewencolon/Psychedelic-Trip-Generator/blob/main/Copy_of_GPT_2_Text_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import pandas as pd
import math
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler
from nltk.tokenize import sent_tokenize
from joblib import load
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from torch.cuda.amp import autocast, GradScaler
from torch.profiler import profile, record_function

# If there's a GPU available...
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


No GPU available, using the CPU instead.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load data
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/modeled.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67516 entries, 0 to 67515
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              67516 non-null  object 
 1   drug               67516 non-null  object 
 2   dosage             67516 non-null  object 
 3   delivery           67516 non-null  object 
 4   weight             67516 non-null  float64
 5   year               67516 non-null  int64  
 6   gender             67516 non-null  object 
 7   report             67516 non-null  object 
 8   processed_report   67516 non-null  object 
 9   mixed              67516 non-null  int64  
 10  drug_category      67516 non-null  object 
 11  report_embeddings  67516 non-null  object 
dtypes: float64(1), int64(2), object(9)
memory usage: 6.2+ MB


In [5]:
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer_path = '/content/drive/MyDrive/Colab Notebooks/Data/'
tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [6]:
unique_drugs = df['drug'].unique()
special_tokens = [f'<{drug}>' for drug in unique_drugs]
tokenizer.add_tokens(special_tokens)

# Create a dictionary mapping each drug to its corresponding special token
special_tokens_dict = {drug: f'<{drug}>' for drug in unique_drugs}

# Apply the mapping to the 'report' column based on the 'drug' column
df['report'] = df.apply(lambda row: special_tokens_dict[row['drug']] + ' ' + row['report'], axis=1)

In [7]:
model.resize_token_embeddings(len(tokenizer))
vocab_size = len(tokenizer)

In [8]:
# First Run
#class GPT2Dataset(Dataset):
#    def __init__(self, txt_list, tokenizer, max_length=1024):
#        self.input_ids = []
#        self.attn_masks = []
#     for txt in txt_list:
#        encodings_dict = tokenizer(txt, truncation=False, padding=False)
#        tokens = encodings_dict['input_ids']
#        self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
#        # Chunk the tokenized text into pieces of max_length
#        for i in range(0, len(tokens), max_length):
#            chunk = tokens[i: i + max_length]
#            attention_mask = [1] * len(chunk)
            # Padding
#            while len(chunk) < max_length:
#                chunk.append(0)
#                attention_mask.append(0)
#            self.input_ids.append(torch.tensor(chunk))
#            self.attn_masks.append(torch.tensor(attention_mask))

In [9]:
input_ids = torch.load('/content/drive/MyDrive/Colab Notebooks/Data/input_ids.pt')
attn_masks = torch.load('/content/drive/MyDrive/Colab Notebooks/Data/attn_masks.pt')

class GPT2Dataset(Dataset):
    def __init__(self, input_ids, attn_masks):
        self.input_ids = input_ids
        self.attn_masks = attn_masks

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

dataset = GPT2Dataset(input_ids, attn_masks)

In [10]:
#reports = df.report.values
#dataset = GPT2Dataset(reports, tokenizer, max_length=1024)

#torch.save(dataset.input_ids, 'D:/Cloud/Google Drive/Colab Notebooks/Data/input_ids.pt')
#torch.save(dataset.attn_masks, 'D:/Cloud/Google Drive/Colab Notebooks/Data/attn_masks.pt')

#tokenizer.save_pretrained('D:/Cloud/Google Drive/Colab Notebooks/Data/')

In [11]:
# Splitting and loading as before
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

num_workers = 2
batch_size = 16
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size, num_workers=num_workers)
validation_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size, num_workers=num_workers)

In [12]:
# Function to generate text
def generate_text(drug, length=500):
    category = get_category_from_embedding(drug)  # Function to categorize the drug based on BigBird embeddings
    input_str = f"The experience of using {drug}, which belongs to the {category} category, is like..."
    inputs = tokenizer.encode(input_str, return_tensors='pt').to(device)
    outputs = model.generate(inputs, max_length=length, num_return_sequences=1, do_sample=True)
    return tokenizer.decode(outputs[0])

In [13]:
#tokenizer.save_pretrained('D:/Cloud/Google Drive/Colab Notebooks/Data/')

In [ ]:
from torch.profiler import profile, record_function

# Define number of epochs
epochs = 3

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * epochs)

accumulation_steps = 4

# Create a profiler context
with profile(activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA], record_shapes=True) as prof:
    for epoch in range(epochs):
        print("Starting epoch", epoch+1)
        model.train()
        print("Model set to training mode")
        optimizer.zero_grad()
        print("Gradients reset")

            for batch_idx, batch in enumerate(train_dataloader):
                print(f"Starting batch {batch_idx}")
            # Optional: use record_function to label sections of your code
            with record_function("Load data"):
                print("Processing batch", batch_idx) # Debug print
                inputs, masks = batch
                print("Data extracted from batch") # Debug print
                inputs, masks = inputs.to(device), masks.to(device)
                print("Data moved to GPU") # Debug print

            with record_function("Forward pass"):
                outputs = model(inputs, attention_mask=masks, labels=inputs)
                loss = outputs.loss

            with record_function("Backward pass"):
                loss.backward()

                if (batch_idx + 1) % accumulation_steps == 0:  # Only update every accumulation_steps
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    scheduler.step()
                    optimizer.zero_grad()

            if batch_idx % 10 == 0:
                processed_percentage = (batch_idx + 1) / len(train_dataloader) * 100
                print(f'\rProcessed: {processed_percentage:.2f}% of data in epoch {epoch + 1}', end='')

        print(f'\nEpoch {epoch + 1}: Training Loss: {loss.item()}')
        sample_text = generate_text("Sample prompt", length=50)
        print(f"Generated Text: {sample_text}")

    print(f"Finished Epoch {epoch+1}")

# Save the trained model
torch.save(model.state_dict(), 'D:/Cloud/Google Drive/Colab Notebooks/Data/trip_reports_model.pth')

# Print profiler results
prof.export_chrome_trace("trace.json")
print(prof.key_averages().table(sort_by="cuda_time_total"))

In [ ]:
# Validation loop
model.eval()
total_eval_loss = 0
eval_steps = 0
with torch.no_grad():
    for batch in validation_dataloader:
        inputs, masks = batch
        inputs, masks = inputs.to(device), masks.to(device)
        outputs = model(inputs, attention_mask=masks, labels=inputs)
        loss = outputs.loss
        total_eval_loss += loss.item()
        eval_steps += 1

avg_val_loss = total_eval_loss / len(validation_dataloader)
perplexity = math.exp(avg_val_loss)  # Compute perplexity from the average loss

print(f'Validation Loss: {avg_val_loss}')
print(f'Validation Perplexity: {perplexity}')

In [ ]:
def generate_report(drug, desired_length_min=300, desired_length_max=500):
    # Step 1: Preprocess
    output_file = 'D:/Cloud/Google Drive/Colab Notebooks/Data/bigbird_embeddings.joblib'
    bigbird_embeddings = load(output_file)
    embedding = bigbird_embeddings[drug]

    # Step 2: Generate text with GPT-2
    text = generate_text(drug)

    # Step 3: Evaluate with RFC (optional)
    # Load the trained Random Forest model
    with open("D:/Cloud/Google Drive/Colab Notebooks/Data/xgb_model.pkl", "rb") as f:
    rfc_model = pickle.load(f)
    vectorized_text = tfidf_vectorizer.transform([text])
    category_prediction = rfc_model.predict(vectorized_text)
    # Validate or modify text based on category prediction if needed

    # Step 4: Post-process text
    sentences = sent_tokenize(text)

    # Concatenate sentences until you reach the desired minimum length
    processed_text = ""
    for sentence in sentences:
        processed_text += sentence + " "
        words = processed_text.split()
        if len(words) >= desired_length_min:
            break

    # If the text exceeds the desired maximum length, truncate it
     words = text.split()
    if len(words) > 4000:
        text = " ".join(words[:4000])
    if len(words) > desired_length_max:
        processed_text = " ".join(words[:desired_length_max])

    return processed_text
